In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using RigidBodyTreeInspector
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using LCPSim
using Polyhedra
using CDDLib
using StaticArrays: SVector
using Gurobi
using JuMP

In [ ]:
mechanism = parse_urdf(Float64, "cartpole.urdf")
world = root_body(mechanism)

basevis = Visualizer()[:cartpole]
delete!(basevis)
vis = basevis[:robot]
setgeometry!(vis, mechanism, parse_urdf("cartpole.urdf", mechanism))

wall_radius = 1.5
μ = 0.5
walls = [planar_obstacle(default_frame(world), [1., 0, 0.], [-wall_radius, 0, 0.], μ), 
    planar_obstacle(default_frame(world), [-1., 0, 0.], [wall_radius, 0, 0.], μ)]
bounds = SimpleHRepresentation(vcat(eye(3), -eye(3)), vcat([wall_radius + 0.1, 0.5, 2.0], -[-wall_radius - 0.1, -0.5, -0.1]))

for wall in walls
    addgeometry!(basevis[:environment], CDDPolyhedron{3, Float64}(intersect(wall.interior, bounds)))
end

pole = findbody(mechanism, "pole")
env = Environment(
    Dict(pole => ContactEnvironment(
            [Point3D(default_frame(pole), SVector(0., 0, 1))],
            walls)))
# env = Environment{Float64}(Dict())

inspect(mechanism, vis)

In [ ]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "slider_to_cart"), [0])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [0])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
u0 = zeros(num_velocities(x0))
xstar = MechanismState(mechanism, q0, v0)

contacts = Point3D[]
Q = diagm([10, 10, 1, 1])
R = 0.1 * eye(num_velocities(x0))
K, S = LCPSim.ContactLQR.contact_lqr(x0, u0, Q, R, contacts)

controller = x -> begin
    -K * (state_vector(x) - state_vector(xstar)) .+ u0
#     u0
end
Δt = 0.01

# set_velocity!(x0, findjoint(mechanism, "slider_to_cart"), [-2])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [π/4])

results = LCPSim.simulate(x0, controller, env, Δt, 200, GurobiSolver(OutputFlag=0));

In [ ]:
set_configuration!(x0, configuration(results[1].state))
settransform!(vis, x0)

In [ ]:
for r in results
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
    for (body, contacts) in r.contacts
        for (i, contact) in enumerate(contacts)
            f = LCPSim.contact_force(contact)
            p = transform_to_root(x0, contact.point.frame) * contact.point
            v = vis[:forces][Symbol(body)][Symbol(i)]
            setgeometry!(v, DrakeVisualizer.PolyLine([p.v, (p + 0.1*f).v]; end_head=DrakeVisualizer.ArrowHead()))
        end
    end
end

In [ ]:
function run_optimization(x0, env, Δt, N; x_nominal=x0, solver=GurobiSolver())
    qstar = copy(configuration(x_nominal))
    vstar = zeros(num_velocities(x_nominal))
    ustar = u0
    
    model, results_opt = LCPSim.optimize(x0, env, Δt, N, Model(solver=solver))
    
    objective = (
        sum(Δt * (r.state.state' * Q * r.state.state + 0.01 * r.input' * R * r.input) for r in results_opt)
       + (results_opt[end].state.state' * S * results_opt[end].state.state)
        )

    for r in results_opt
        for (joint, jrs) in r.joint_contacts
            for joint_result in jrs
#                 objective += joint_result.λ^2
            end
        end 
    end
    
    @objective model Min objective

    controller = x -> begin
        -K * (state_vector(x) - vcat(qstar, vstar)) .+ ustar
    end
    
#     for r in results_opt
#         @constraint model r.input .== -K * r.state.state
#     end
    
    Δt_sim = 0.01
    time_ratio = convert(Int, Δt / Δt_sim)
    results = LCPSim.simulate(x0, controller, env, Δt_sim, time_ratio * N, GurobiSolver(OutputFlag=0))
    if length(results) == length(results_opt)
        setvalue.(results_opt, results[1:time_ratio:end])
        ConditionalJuMP.warmstart!(model, false)
    end
    solve(model)
    getvalue.(results_opt)
end

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
# set_configuration!(x0, findjoint(mechanism, "slider_to_cart"), [-1.5])
# set_velocity!(x0, findjoint(mechanism, "slider_to_cart"), [-2])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [π/4])
Δt = 0.01
gap = 1e-3
timelimit = 60
horizon = 30

r_control = []
x_control = MechanismState(mechanism, copy(q0), copy(v0))
controller = x -> begin
    set_velocity!(x_control, velocity(x))
    set_configuration!(x_control, configuration(x))
    r_control = run_optimization(x_control, env, Δt, horizon; x_nominal=xstar, solver=GurobiSolver(OutputFlag=0, MIPGap=gap, TimeLimit=timelimit))
    u = r_control[1].input
    settransform!(vis, x_control)
#     for r in r_control
#         set_configuration!(x0, configuration(r.state))
#         settransform!(vis, x0)
#     end
    u
    
    @show u
end

results_mpc = LCPSim.simulate(x0, controller, env, Δt, 200, GurobiSolver(OutputFlag=0));

In [ ]:
for r in results_mpc
    sleep(Δt)
    set_configuration!(x0, configuration(r.state))
    settransform!(vis, x0)
    for (body, contacts) in r.contacts
        for (i, contact) in enumerate(contacts)
            f = LCPSim.contact_force(contact)
            p = transform_to_root(x0, contact.point.frame) * contact.point
            v = vis[:forces][Symbol(body)][Symbol(i)]
            setgeometry!(v, DrakeVisualizer.PolyLine([p.v, (p + 0.1*f).v]; end_head=DrakeVisualizer.ArrowHead()))
        end
    end
end

In [ ]:
set_configuration!(x0, configuration(results_mpc[end].state))
set_velocity!(x0, velocity(results_mpc[end].state))

r_control = []
x_control = MechanismState(mechanism, copy(q0), copy(v0))
controller = x -> begin
    set_velocity!(x_control, velocity(x))
    set_configuration!(x_control, configuration(x))
    r_control = run_optimization(x_control, env, Δt, horizon; x_nominal=xstar, solver=GurobiSolver(OutputFlag=0, MIPGap=gap, TimeLimit=timelimit))
    u = r_control[1].input
    @show u
end

append!(results_mpc, LCPSim.simulate(x0, controller, env, Δt, 100, GurobiSolver(OutputFlag=0)));